In [7]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [2]:
!pip install pandas

  Using cached numpy-1.26.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (115 kB)
  Using cached pytz-2023.3.post1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 76.3 MB/s eta 0:00:0000:010:01
Using cached numpy-1.26.2-cp311-cp311-macosx_11_0_arm64.whl (14.0 MB)
Using cached pytz-2023.3.post1-py2.py3-none-any.whl (502 kB)


In [4]:
!pip install tensorflow

  Using cached tensorflow-2.15.0-cp311-cp311-macosx_12_0_arm64.whl.metadata (3.6 kB)
  Using cached tensorflow_macos-2.15.0-cp311-cp311-macosx_12_0_arm64.whl.metadata (4.2 kB)
  Using cached absl_py-2.0.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.5.4-py3-none-any.whl (19 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached h5py-3.10.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (2.5 kB)
  Using cached libclang-16.0.6-py2.py3-none-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached ml_dtypes-0.2.0-cp311-cp311-macosx_10_9_universal2.whl.metadata (20 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached tensorflow_io_gcs_filesystem-0.34.0-cp311-cp311-macosx_12_0_arm64.whl.metadata (14 kB)
  Using cached grpcio-1.60.0-cp311-cp

In [6]:
!pip install scikeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.4/165.4 kB 5.5 MB/s eta 0:00:00
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.2.0-py3-none-any.whl.metadata (10.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 63.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 69.9 MB/s eta 0:00:00
Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 79.5 MB/s eta 0:00:0000:01:00:01
Using cached threadpoolctl-3.2.0-py3-none-any.whl (15 kB)


In [9]:
data = pd.read_csv('datasets/Tweets.csv')

In [10]:
data = data[['text','airline_sentiment']]

In [11]:
data['text'].values

array(['@VirginAmerica What @dhepburn said.',
       "@VirginAmerica plus you've added commercials to the experience... tacky.",
       "@VirginAmerica I didn't today... Must mean I need to take another trip!",
       ...,
       '@AmericanAir Please bring American Airlines to #BlackBerry10',
       "@AmericanAir you have my money, you change my flight, and don't answer your phones! Any other suggestions so I can make my commitment??",
       '@AmericanAir we have 8 ppl so we need 2 know how many seats are on the next flight. Plz put us on standby for 4 people on the next flight?'],
      dtype=object)

In [12]:
data = data[data.airline_sentiment != "neutral"].copy()

In [15]:
import nltk
from nltk import PorterStemmer

In [16]:
import re
import string

In [14]:
!pip install nltk

  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.1 MB/s eta 0:00:00
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.0/291.0 kB 8.1 MB/s eta 0:00:00
Using cached click-8.1.7-py3-none-any.whl (97 kB)
Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)


In [17]:
# Cleaning Text

def clean_text(txt):

          """
          removing all hashtags , punctuations, stop_words  and links, also stemming words
          """
          from nltk.corpus import stopwords
          txt = txt.lower()
          txt = re.sub(r"(?<=\w)nt", "not",txt) #change don't to do not cna't to cannot
          txt = re.sub(r"(@\S+)", "", txt)  # remove hashtags
          txt = re.sub(r'\W', ' ', str(txt)) # remove all special characters including apastrophie
          txt = txt.translate(str.maketrans('', '', string.punctuation)) # remove punctuations
          txt = re.sub(r'\s+[a-zA-Z]\s+', ' ', txt)   # remove all single characters (it's -> it s then we need to remove s)
          txt = re.sub(r'\s+', ' ', txt, flags=re.I) # Substituting multiple spaces with single space
          txt = re.sub(r"(http\S+|http)", "", txt) # remove links
          txt = ' '.join([PorterStemmer().stem(word=word) for word in txt.split(" ") if word not in stopwords.words('english') ]) # stem & remove stop words
          txt = ''.join([i for i in txt if not i.isdigit()]).strip() # remove digits ()
          return txt

In [18]:
data['text'] = data['text'].apply(clean_text)

In [19]:
data['text'].values

array(['plu ad commerci experi tacki',
       'realli aggress blast obnoxi enotertainmenot guest face amp littl recours',
       'realli big bad thing', ..., 'thank got differenot flight chicago',
       'leav  minut late flight warn commun unotil  minut late flight call shitti custom svc',
       'money chang flight answer phone suggest make commitmenot'],
      dtype=object)

In [21]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [28]:
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split

In [34]:
import numpy as np

In [22]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

# pad: to make all input of same length
X = pad_sequences(X)

In [25]:
X.shape

(11541, 21)

In [27]:
embed_dim = 128
lstm_out = 196
input_len = 21
import time
model = Sequential()
model.add(Embedding(max_fatures, embed_dim, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))

In [29]:
start=time.time()
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print("Time to compile model:",time.time()-start)

Time to compile model: 0.02385401725769043


In [30]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 21, 128)           256000    
                                                                 
 spatial_dropout1d (Spatial  (None, 21, 128)           0         
 Dropout1D)                                                      
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 2)                 394       
                                                                 
Total params: 511194 (1.95 MB)
Trainable params: 511194 (1.95 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [31]:
Y = pd.get_dummies(data['airline_sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.15, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(9809, 21) (9809, 2)
(1732, 21) (1732, 2)


In [32]:
from tqdm import tqdm
batch_size = 32
tqdm(model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2))

Epoch 1/7
307/307 - 20s - loss: 0.3329 - accuracy: 0.8652 - 20s/epoch - 64ms/step
Epoch 2/7
307/307 - 20s - loss: 0.1875 - accuracy: 0.9242 - 20s/epoch - 65ms/step
Epoch 3/7
307/307 - 18s - loss: 0.1549 - accuracy: 0.9385 - 18s/epoch - 59ms/step
Epoch 4/7
307/307 - 18s - loss: 0.1365 - accuracy: 0.9464 - 18s/epoch - 57ms/step
Epoch 5/7
307/307 - 19s - loss: 0.1182 - accuracy: 0.9526 - 19s/epoch - 61ms/step
Epoch 6/7
307/307 - 18s - loss: 0.1063 - accuracy: 0.9598 - 18s/epoch - 58ms/step
Epoch 7/7
307/307 - 18s - loss: 0.0950 - accuracy: 0.9614 - 18s/epoch - 58ms/step


0it [00:00, ?it/s]

In [33]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]

score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

55/55 - 1s - loss: 0.2685 - accuracy: 0.9070 - 647ms/epoch - 12ms/step
score: 0.27
acc: 0.91


In [35]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 0)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1

In [36]:
print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 75.53191489361703 %
neg_acc 93.76026272577997 %


In [51]:
twt = ['americanair leaving over 20 minutes late flight no warnings or communication until we were 15 minutes late flight thats called shitty customer svc']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt.shape)
sentiment = model.predict(twt,batch_size=1,verbose = 0)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

(1, 28)
negative


In [62]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 21, 128)           256000    
                                                                 
 spatial_dropout1d (Spatial  (None, 21, 128)           0         
 Dropout1D)                                                      
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 2)                 394       
                                                                 
Total params: 511194 (1.95 MB)
Trainable params: 511194 (1.95 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [38]:
import pickle

#saving model
model.save("models/binaryClassificationModel.h5")

# saving tokenizer
with open('models/tokenizerBinaryClassification.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

/Users/ranjeetnagarkar/Desktop/twitter_llm_project/.venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [145]:

class BinaryInference:

      def __init__(self):
          self.load_models()

      def get_model(self):
          max_fatures = 2000
          embed_dim = 128
          lstm_out = 196
          input_len = 21
          model = Sequential()
          model.add(Embedding(max_fatures, embed_dim,input_length = input_len))
          model.add(SpatialDropout1D(0.4))
          model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
          model.add(Dense(2,activation='softmax'))
          model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
          return model

      def clean_text(self, txt):

          """
          removing all hashtags , punctuations, stop_words  and links, also stemming words
          """
          from nltk.corpus import stopwords
          txt = txt.lower()
          txt = re.sub(r"(?<=\w)nt", "not",txt) #change don't to do not cna't to cannot
          txt = re.sub(r"(@\S+)", "", txt)  # remove hashtags
          txt = re.sub(r'\W', ' ', str(txt)) # remove all special characters including apastrophie
          txt = txt.translate(str.maketrans('', '', string.punctuation)) # remove punctuations
          txt = re.sub(r'\s+[a-zA-Z]\s+', ' ', txt)   # remove all single characters (it's -> it s then we need to remove s)
          txt = re.sub(r'\s+', ' ', txt, flags=re.I) # Substituting multiple spaces with single space
          txt = re.sub(r"(http\S+|http)", "", txt) # remove links
          txt = ' '.join([PorterStemmer().stem(word=word) for word in txt.split(" ") if word not in stopwords.words('english') ]) # stem & remove stop words
          txt = ''.join([i for i in txt if not i.isdigit()]).strip() # remove digits ()
          return txt

      def load_models(self):
          with open('models/tokenizerBinaryClassification.pickle', 'rb') as handle:
              self.tokenizer = pickle.load(handle)

          self.model = self.get_model()
          self.model.load_weights("models/binaryClassificationModel.h5")


      def predict_complaint(self, text):
          #vectorizing the tweet by the pre-fitted tokenizer instance
          text = self.clean_text(text)
          twt = self.tokenizer.texts_to_sequences([text])
          #padding the tweet to have exactly the same shape as `embedding_2` input
          twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
          self.model.predict(twt[:,:21],batch_size=1,verbose = 0)
          complain = self.model.predict(twt,batch_size=1,verbose = 0)[0]
          if(np.argmax(complain) == 0):
              print("negative")
              return True
          elif (np.argmax(complain) == 1):
              print("positive")
              return False

In [146]:
bi = BinaryInference()

In [148]:
bi.predict_complaint("@americanair leaving over 20 minutes late flight no warnings or communication until we were 15 minutes late flight thats called shitty customer svc")
bi.predict_complaint("americanair great service")

negative
positive


False

In [144]:
dummy_twt = [[0] * 21]
dummy_twt

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [131]:
max_fatures = 2000
embed_dim = 128
lstm_out = 196
input_len = 21
import time
model2 = Sequential()
model2.add(Embedding(max_fatures, embed_dim, input_length = input_len))
model2.add(SpatialDropout1D(0.4))
model2.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model2.add(Dense(2, activation='softmax'))
model2.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
model2.build(input_shape=(None, 21))


In [132]:
def clean_text(txt):
    """
    removing all hashtags , punctuations, stop_words  and links, also stemming words
    """
    from nltk.corpus import stopwords
    txt = txt.lower()
    txt = re.sub(r"(?<=\w)nt", "not",txt) #change don't to do not cna't to cannot
    txt = re.sub(r"(@\S+)", "", txt)  # remove hashtags
    txt = re.sub(r'\W', ' ', str(txt)) # remove all special characters including apastrophie
    txt = txt.translate(str.maketrans('', '', string.punctuation)) # remove punctuations
    txt = re.sub(r'\s+[a-zA-Z]\s+', ' ', txt)   # remove all single characters (it's -> it s then we need to remove s)
    txt = re.sub(r'\s+', ' ', txt, flags=re.I) # Substituting multiple spaces with single space
    txt = re.sub(r"(http\S+|http)", "", txt) # remove links
    txt = ' '.join([PorterStemmer().stem(word=word) for word in txt.split(" ") if word not in stopwords.words('english') ]) # stem & remove stop words
    txt = ''.join([i for i in txt if not i.isdigit()]).strip() # remove digits ()
    return txt

In [133]:
with open('models/tokenizerBinaryClassification.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

#text = "americanair leaving over 20 minutes late flight no warnings or communication until we were 15 minutes late flight thats called shitty customer svc"
text = "americanair great service"
text = clean_text(text)
twt = tokenizer.texts_to_sequences([text])
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)

print(twt)

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0 49  7]]


In [134]:
model2.load_weights("models/binaryClassificationModel.h5")

In [135]:
#complain = model2.predict(twt[:,:21],batch_size=1,verbose = 0)[0]
complain = model2.predict(twt,batch_size=1,verbose = 0)[0]
print(complain)

ValueError: in user code:

    File "/Users/ranjeetnagarkar/Desktop/twitter_llm_project/.venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "/Users/ranjeetnagarkar/Desktop/twitter_llm_project/.venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/ranjeetnagarkar/Desktop/twitter_llm_project/.venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/ranjeetnagarkar/Desktop/twitter_llm_project/.venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 2381, in predict_step
        return self(x, training=False)
    File "/Users/ranjeetnagarkar/Desktop/twitter_llm_project/.venv/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70, in e

In [126]:
if(np.argmax(complain) == 0):
    print("negative")
elif (np.argmax(complain) == 1):
    print("positive")

positive


In [106]:
complain = model.predict(twt,batch_size=1,verbose = 0)[0]
if(np.argmax(complain) == 0):
    print("negative")
elif (np.argmax(complain) == 1):
    print("positive")

positive
